In [3]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from pathlib import Path
from read_roi import read_roi_zip
from collections import OrderedDict, Counter
import math
import PIL
from PIL import Image
from PIL.TiffTags import TAGS
import tkinter as tk
from tkinter import filedialog
import seaborn as sns

In [4]:
askdirectory = filedialog.askdirectory() # show an "Open" dialog box and return the path to the selected file
path = Path(askdirectory)
path

WindowsPath('C:/Users/Brouhard Lab/Data/Images/2022-09-07_dynamics/2022-09-07_dynamics_Ch1-tub10uM')

In [5]:
tubulin = '[Tubulin] ' r'$(\mu M)$'
tub = 'tub'

investigator = 'Brandi'

In [6]:
dateloc = str(path).find("202")
date = str(path)[dateloc : dateloc+10]

folders = [x for x in path.iterdir() if x.is_dir()]
folders = [i for i in folders if (str(i)[-5:] == 'final') == True]

ch = []
conc_tub = []

for i in folders:
    CHloc = str(i).rfind("Ch")
    tubloc = str(i).rfind(tub)
    ch = ch + [str(i)[CHloc+2:CHloc+3]]
    conc_tub = conc_tub + [(str(i)[tubloc+3:tubloc+5])]

date,ch, conc_tub

('2022-09-07', ['1'], ['10'])

In [7]:
tub_conc =  [float(i) for i in conc_tub]
frame_analyzer = pd.read_csv(path/'frame_rates_avg.csv',sep=',')
pixel =  0.107 #T2 0.107 #T1 0.0633
folders

[WindowsPath('C:/Users/Brouhard Lab/Data/Images/2022-09-07_dynamics/2022-09-07_dynamics_Ch1-tub10uM/2022-09-07_dynamics_Ch1-tub10uM_final')]

In [8]:
#frame_analyzer = frame_analyzer.iloc[[1,3]].reset_index()
frame_analyzer

,Directory,Mean,SD,Frames,Outliers,MeanWOutliers,SDWOutliers
0,C:\Users\Brouhard Lab\Data\Images\2022-09-07_d...,5.01,0.25,405,4,12.0,127.82


## Line Dynamics

In [9]:
def open_roi_line(k,name):
    roi_path = folders[k]/(name)
    roi_data = read_roi_zip(roi_path)
    roi_df = pd.DataFrame(columns=['Track ID','x1','x2','y1','y2'])
    
    for key in roi_data:
        roi_df1 = pd.DataFrame({'Track ID': [roi_data[key]['name']],'x1': [roi_data[key]['x1']],'x2': [roi_data[key]['x2']],'y1': [roi_data[key]['y1']],'y2': [roi_data[key]['y2']]})
        roi_df = pd.concat([roi_df,roi_df1])
        
    roi_df = roi_df.reset_index(drop = True)
    return roi_df

In [10]:
open_roi_line(0,'kymoslope_line.zip')

,Track ID,x1,x2,y1,y2
0,0085-0071,31.0,110.0,36.0,134.0
1,0211-0067,32.0,101.0,154.0,268.0
2,0322-0113,60.0,165.0,247.0,397.0
3,0100-0081,60.0,102.0,68.0,132.0
4,0177-0069,54.0,84.0,149.0,204.0
...,...,...,...,...,...
100,0095-0090,28.0,151.0,16.0,174.0
101,0261-0055,21.0,89.0,193.0,328.0
102,0039-0053,36.0,70.0,16.0,61.0
103,0202-0125,41.0,209.0,75.0,328.0


In [14]:
roi_path = folders[0]/('start_line.zip')
roi_data = read_roi_zip(roi_path)
roi_data

OrderedDict([('0001-0075',
              {'type': 'line',
               'x1': 43.0,
               'x2': 107.0,
               'y1': 1.0,
               'y2': 0.0,
               'draw_offset': False,
               'width': 0,
               'name': '0001-0075',
               'position': 0}),
             ('0001-0147',
              {'type': 'line',
               'x1': 121.0,
               'x2': 173.0,
               'y1': 1.0,
               'y2': 0.0,
               'draw_offset': False,
               'width': 0,
               'name': '0001-0147',
               'position': 0})])

In [15]:
Length = 'Length ' r'$(\mu m)$'
GrowthRate = 'Growth Rate ' r'$(\mu m / min)$'
Lifetime = 'Lifetime ' r'$(min)$'

In [16]:
def line_dynamics(k):
        
    data = open_roi_line(k,'kymoslope_line.zip')
    supp = open_roi_line(k,'start_line.zip')
    
    
    spf = frame_analyzer['Mean'][k];
    frames = frame_analyzer['Frames'][k]-2
    
    suppn = len(supp)
    start = sum(abs(supp['y2']-supp['y1']))/suppn;
    
    n = len(data)
    pre_df = {'Date': np.full((n), date),'Ch': np.full((n), ch[k]),'Concentration ' r'$(\mu M)$': np.full((n), tub_conc[k])}
    df = pd.DataFrame(pre_df) 
    
    
    df[Length] = abs((data['x2']-data['x1'])*pixel)
    df[Lifetime] = abs((data['y2']-data['y1'])*spf/60)
    
    df[GrowthRate] = df[Length]/df[Lifetime]
    df[GrowthRate] = df[GrowthRate].apply(lambda x: x if x <= 5 else np.NaN )
    
    data[Lifetime] = abs((data['y2']-data['y1'])*spf/60)
    df[Lifetime] = data[[Lifetime,'y2']].apply(lambda x: x[Lifetime] if x['y2'] < frames else np.NaN, axis=1)
    
    df[Length] = df[GrowthRate]*df[Lifetime]
    
    df.loc[pd.isnull(df[GrowthRate]), [Length,
        Lifetime]] = np.NaN
    
    df = df[['Date','Ch','Concentration ' r'$(\mu M)$',Length,
             Lifetime,GrowthRate]]

    
    return df

In [17]:
line_dynamics(0)

,Date,Ch,Concentration $(\mu M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$
0,2022-09-07,5,8.0,10.379,12.299000,0.843890
1,2022-09-07,5,8.0,6.206,6.191333,1.002369
2,2022-09-07,5,8.0,11.021,12.884667,0.855358
3,2022-09-07,5,8.0,3.210,3.932333,0.816309
4,2022-09-07,5,8.0,4.708,4.936333,0.953744
...,...,...,...,...,...,...
61,2022-09-07,5,8.0,4.066,4.769000,0.852590
62,2022-09-07,5,8.0,12.519,11.546000,1.084272
63,2022-09-07,5,8.0,3.424,4.016000,0.852590
64,2022-09-07,5,8.0,8.346,9.119667,0.915165


In [18]:
path.parents[0],path

(WindowsPath('C:/Users/Brouhard Lab/Data/Images/2022-09-07_dynamics'),
 WindowsPath('C:/Users/Brouhard Lab/Data/Images/2022-09-07_dynamics/2022-09-07_dynamics_Ch5-tub08uM'))

In [ ]:
# newdirectory = str(path.parents[0])+'//Data_Sheets'
# newmydir = Path(newdirectory)
# newmydir.mkdir(exist_ok=True)

In [19]:
df = line_dynamics(0)
for i in range(1,len(ch)):
    df0 = line_dynamics(i)
    df = pd.concat([df,df0], axis=0)

df.to_csv(path/(date+'_Ch1_results.csv'), encoding='utf-8', index=False)
#df.to_csv(newmydir/(investigator+'_'+date+'.csv'), encoding='utf-8', index=False)
df

,Date,Ch,Concentration $(\mu M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$
0,2022-09-07,5,8.0,10.379,12.299000,0.843890
1,2022-09-07,5,8.0,6.206,6.191333,1.002369
2,2022-09-07,5,8.0,11.021,12.884667,0.855358
3,2022-09-07,5,8.0,3.210,3.932333,0.816309
4,2022-09-07,5,8.0,4.708,4.936333,0.953744
...,...,...,...,...,...,...
61,2022-09-07,5,8.0,4.066,4.769000,0.852590
62,2022-09-07,5,8.0,12.519,11.546000,1.084272
63,2022-09-07,5,8.0,3.424,4.016000,0.852590
64,2022-09-07,5,8.0,8.346,9.119667,0.915165


In [174]:
dfmean = df.groupby('Ch').mean()
#dfmean.to_csv(path/(date+'_0_mean.csv'), encoding='utf-8', index=False)
dfmean

,Concentration $(\mu M)$,Length $(\mu m)$,Lifetime $(min)$,Growth Rate $(\mu m / min)$
Ch,,,,
1,10.0,7.456371,8.540062,0.897054


...

## Join Data

In [ ]:
askdirectory = filedialog.askdirectory() # open master folder for combining data
path = Path(askdirectory)

In [ ]:
files = [x for x in path.iterdir() if x.is_file()]
files = [i for i in files if (str(i)[-10:] != 'results.csv') == True]
files = [i for i in files if (str(i)[-3:] == 'csv') == True]
files

In [ ]:
data = pd.read_csv(files[0], encoding='utf-8')
for i in range(1,len(files)):
    d = pd.read_csv(files[i], encoding='utf-8')
    data = pd.concat([data,d])
    # data = data.append(d, ignore_index=True)
data[Type].unique()

In [ ]:
data # check concat function

In [ ]:
data.to_csv(path/(date+'_MasterSheet.csv'), encoding='utf-8', index=False)

In [ ]:
askmastersheet = filedialog.askopenfilename() # show an "Open" dialog box and return the path to the selected file
data = pd.read_csv(askmastersheet, encoding='utf-8')
data

In [ ]:
pathmastersheet = Path(askmastersheet)
resultdirectory = str(pathmastersheet.parents[0])+'//Plots'
newmydir = Path(resultdirectory)
newmydir.mkdir(exist_ok=True)
path = newmydir
path